In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 

import xml.etree.ElementTree as ET
import glob
import json
import shutil


remove invalid files

In [ ]:
# rename to jpg
input_dir = "/home/jyothir-dr/Downloads/vehicles/train/Final Train Dataset/"
ext = [".png", ".JPG", ".PNG"]

files = os.listdir(input_dir)
print(len(files))
for fil in files:
    for x in ext:
        try:
            if fil.endswith(x): 
                os.rename(os.path.join(input_dir,fil), os.path.join(input_dir,os.path.basename(fil)[:-4]+".jpg"))
                # print(os.path.join(input_dir,fil), os.path.join(input_dir,os.path.basename(fil)[:-4]+".jpg"))
            elif fil.endswith(".jpeg"): #str(x) for x in ext):
                os.rename(os.path.join(input_dir,fil), os.path.join(input_dir, os.path.basename(fil)[:-5]+".jpg"))
                # print(os.path.join(input_dir,fil), os.path.join(input_dir, os.path.basename(fil)[:-5]+".jpg"))
        except:
            FileNotFoundError

In [ ]:
# remove wrong data
classes = ["car", "bus", "motorbike"]
def remove_wrong_annot (lbl_path, wrg_fol):
    files = glob.glob(os.path.join(lbl_path, '*.xml'))
    annotations = [os.path.join(lbl_path, f) for f in files if f.endswith('.xml')]
    data_list = []
    for file_path in annotations:
        try:
            root = ET.parse(file_path).getroot()
            data = {}
            # if cv2.imread(str(Path(str(datadir) + '/'+ root.find("./filename").text))):
            if os.path.basename(root.find("./filename").text).split('.')[:-1] != os.path.basename(file_path).split('.')[:-1]:
                if os.path.basename(root.find("./filename").text.replace("\t", "  ")).split('.')[:-1]!= os.path.basename(file_path).split('.')[:-1]:
                    shutil.move(file_path, os.path.join(wrg_fol, os.path.basename(file_path)))
                    try:
                        shutil.move(str(lbl_path+os.path.basename(file_path).split('.')[:-1][0]+".jpg"),
                                        str(os.path.join(wrg_fol, os.path.basename(file_path).split('.')[:-1][0] + ".jpg")))
                    except:
                        FileExistsError
        except:
            pass


In [ ]:
wrg_fol = "/home/jyothir-dr/Downloads/vehicles/train/wrong"
remove_wrong_annot(input_dir, wrg_fol)

In [ ]:
classes = ["car", "bus", "motorbike"]
idx_list = [1, 2, 3]
datadir = "/home/jyothir-dr/Downloads/vehicles/train/Final Train Dataset/"

def generate_df (lbl_path):
    files = glob.glob(os.path.join(lbl_path, '*.xml'))
    annotations = [os.path.join(lbl_path, f) for f in files if f.endswith('.xml')]
    print("length of anno = ", len(annotations))
    data_list = []
    for file_path in annotations:
        try:
            root = ET.parse(file_path).getroot()
            data = {}
            # if cv2.imread(str(Path(str(datadir) + '/'+ root.find("./filename").text))):
            if os.path.exists(str((lbl_path) + '/'+ root.find("./filename").text)):

                if root.find("./object/name").text in classes:
                    index = classes.index(root.find("./object/name").text)
                    data['filename'] = (str(lbl_path) + '/'+ root.find("./filename").text)
                    data['width'] = root.find("./size/width").text
                    data['height'] = root.find("./size/height").text
                    data['class'] = int(index)+1
                    data['xmin'] = int(root.find("./object/bndbox/xmin").text)
                    data['ymin'] = int(root.find("./object/bndbox/ymin").text)
                    data['xmax'] = int(root.find("./object/bndbox/xmax").text)
                    data['ymax'] = int(root.find("./object/bndbox/ymax").text)
                    data_list.append(data)
        except:
            pass
        # if root:

    return pd.DataFrame(data_list)

In [ ]:
train_df = generate_df(datadir)
print(train_df.shape)
train_df
train_df.to_pickle("/home/jyothir-dr/Downloads/vehicles/train/3class_df.pkl")  

In [ ]:
train_df['class'].isnull().values.any(), train_df['class'].value_counts()

In [ ]:
train_df['class'].value_counts().sort_index().plot(kind='bar')

In [ ]:
# spli data
save_dir = "/home/jyothir-dr/Downloads/vehicles/train/"
img_list = []
xml_list = []
import random
random.seed(111)

def make_dir(path1):
    if not os.path.isdir(path1):
        os.makedirs(path1)
        
def list_dir (path):
    img_list =[]
    file_list = os.listdir(path)
    for x in file_list:
        if x.endswith(".jpg"):
            img_list.append(x)
        else:
            xml_list.append(x)
            # print(x)
    return img_list

def split_data(img_list, fol_name, test_num):
    make_dir(os.path.join(save_dir, fol_name))
    print(len(img_list), test_num)
    random.shuffle(img_list)
    test_imgs = img_list[:test_num]
    test_data = [os.path.join(datadir,  x) for x in test_imgs]
    test_labels = [os.path.join(datadir,  x.replace(".jpg", ".xml")) for x in test_imgs]
    # print(test_labels)

    for file in test_data:
        # print(file)
        shutil.move(file, os.path.join(save_dir, fol_name))
    for file in test_labels:
        # print("l:", file)
        shutil.move(file, os.path.join(save_dir, fol_name))
    if fol_name == "val":
        train_dir = os.path.join(save_dir, "train")
        os.rename(datadir, train_dir)
        

In [ ]:
img_list = list_dir(datadir)
total = len(img_list)
test_num=int(np.round(total*0.15))
print("Total number:",total,"\n","Test number:",test_num,"\n","Val number:",test_num)
split_data(img_list, "test", test_num)

img_list2 = list_dir(datadir)
split_data(img_list2, "val", test_num)


In [15]:
from utils import create_json, get_label2id
label2id = get_label2id(labels_path="labels.txt")
save_dir = "/home/jyothir-dr/Downloads/vehicles/train"
dir_path = "/home/jyothir-dr/Downloads/vehicles/train/train"
output_jsonpath = os.path.join(save_dir, 'annotation_train.json')
create_json(label2id, dir_path, output_jsonpath)
dir_path = "/home/jyothir-dr/Downloads/vehicles/train/test"
output_jsonpath = os.path.join(save_dir, 'annotation_test.json')
create_json(label2id, dir_path, output_jsonpath)
dir_path = "/home/jyothir-dr/Downloads/vehicles/train/val"
output_jsonpath = os.path.join(save_dir, 'annotation_val.json')
create_json(label2id, dir_path, output_jsonpath)


labels_ids:  [1, 2, 3]
Start converting !


100%|██████████| 1966/1966 [00:00<00:00, 4702.76it/s]


Start converting !


100%|██████████| 421/421 [00:00<00:00, 3336.88it/s]


Start converting !


100%|██████████| 421/421 [00:00<00:00, 3194.37it/s]


In [14]:
import cv2
input_dir = "/home/jyothir-dr/Downloads/vehicles/train/test"
files = os.listdir(input_dir)
print(len(files))
for fil in files:
    if fil.endswith(".jpg"):
        img = cv2.imread(os.path.join(input_dir,os.path.basename(fil)), 0)
        img2 = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        cv2.imwrite(os.path.join(input_dir ,os.path.basename(fil)), img2)


842
